In [ ]:
import torch
import torch.nn as nn
import numpy

In [ ]:
class conv_block(nn.Module):
    def __init__(self, inputs, num_filters):
        super(conv_block,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding="same")
        self.inputs = inputs
    def forward(inputs):
        

In [ ]:
class MyUnet(nn.Module):
    def __init__(self, )